# Introduction

This notebook is created to run this repo on a single gpu machine.

In [10]:
import sys
import os

BASE_DIR = os.getcwd().rsplit("/",1)[0]
sys.path.append(BASE_DIR)
print("adding Base Dir Path", BASE_DIR)


from torch.utils.data.dataloader import DataLoader
from pytorch_lightning import Trainer
from monoscene.models.monoscene import MonoScene
from monoscene.data.NYU.nyu_dm import NYUDataModule, collate_fn, NYUDataset
from monoscene.data.semantic_kitti.kitti_dm import KittiDataModule
from monoscene.data.kitti_360.kitti_360_dm import Kitti360DataModule
import hydra
from omegaconf import DictConfig
import torch
import numpy as np
import os
from hydra.utils import get_original_cwd
from tqdm import tqdm
import pickle




adding Base Dir Path /monoscene/MonoScene


# Config

In [11]:
import yaml
from types import SimpleNamespace

with open("../monoscene/config/monoscene.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
        config = SimpleNamespace(**config)
    except yaml.YAMLError as exc:
        print(exc)

In [12]:
config

namespace(dataset='kitti_360',
          n_relations=4,
          enable_log=False,
          kitti_root='/path/to/semantic_kitti',
          kitti_preprocess_root='/path/to/kitti/preprocess/folder',
          kitti_logdir='/path/to/semantic_kitti/logdir',
          NYU_root='/monoscene/MonoScene/nyu/NYU_dataset/depthbin/',
          NYU_preprocess_root='/monoscene/MonoScene/nyu/NYU_dataset/depthbin/preprocess',
          logdir='/monoscene/logdir',
          output_path='/path/to/output',
          fp_loss=True,
          frustum_size=8,
          batch_size=1,
          n_gpus=0,
          num_workers_per_gpu=0,
          exp_prefix='exp',
          run=1,
          lr='1e-4',
          weight_decay='1e-4',
          context_prior=True,
          relation_loss=True,
          CE_ssc_loss=True,
          sem_scal_loss=True,
          geo_scal_loss=True,
          project_1_2=True,
          project_1_4=True,
          project_1_8=True)

# Data Loader

In [13]:


project_scale = 1
feature = 200
full_scene_size = (60, 36, 60)

test_ds = NYUDataset(
            split="test",
            preprocess_root=config.NYU_preprocess_root,
            n_relations=4,
            root=config.NYU_root,
            frustum_size=4,
            fliplr=0.0,
            color_jitter=None,
        )

data_loader = DataLoader(
            test_ds,
            batch_size=3,
            drop_last=False,
            shuffle=False,
            pin_memory=True,
            collate_fn=collate_fn,
        )




# Loading Model

In [14]:
model_path = "/monoscene/MonoScene/trained_models/monoscene_nyu.ckpt"
model = MonoScene.load_from_checkpoint(
        model_path,
        feature=feature,
        project_scale=project_scale,
        fp_loss=config.fp_loss,
        full_scene_size=full_scene_size,
    )
model.cuda()
model.eval()

n_relations 4
Loading base model ()...

Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Done.
Removing last two layers (global_pool & classifier).
Building Encoder-Decoder model..Done.


MonoScene(
  (projects): ModuleDict(
    (1): FLoSP()
    (2): FLoSP()
    (4): FLoSP()
    (8): FLoSP()
  )
  (net_3d_decoder): UNet3D(
    (process_1_4): Sequential(
      (0): Process(
        (main): Sequential(
          (0): Bottleneck3D(
            (conv1): Conv3d(200, 50, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (bn1): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv3d(50, 50, kernel_size=(1, 1, 3), stride=(1, 1, 1), padding=(0, 0, 1), bias=False)
            (bn2): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv3d(50, 50, kernel_size=(1, 3, 1), stride=(1, 1, 1), padding=(0, 1, 0), bias=False)
            (bn3): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv4): Conv3d(50, 50, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
            (bn4): BatchNorm3d(50, eps=1e-05

# Inference

In [ ]:
output_path = '/monoscene/Monoscene/outputs/'
with torch.no_grad():
    for batch in tqdm(data_loader):
        batch["img"] = batch["img"].cuda()
        pred = model(batch)
        y_pred = torch.softmax(pred["ssc_logit"], dim=1).detach().cpu().numpy()
        y_pred = np.argmax(y_pred, axis=1)
        for i in range(config.batch_size):
            out_dict = {"y_pred": y_pred[i].astype(np.uint16)}
            if "target" in batch:
                out_dict["target"] = (
                    batch["target"][i].detach().cpu().numpy().astype(np.uint16)
                )
            
            # NYU write Path
            write_path = output_path
            filepath = os.path.join(write_path, batch["name"][i] + ".pkl")
            out_dict["cam_pose"] = batch["cam_pose"][i].detach().cpu().numpy()
            out_dict["vox_origin"] = (
                batch["vox_origin"][i].detach().cpu().numpy()
            )
            
            os.makedirs(write_path, exist_ok=True)
            with open(filepath, "wb") as handle:
                pickle.dump(out_dict, handle)
                print("wrote to", filepath)

  0%|          | 1/218 [00:02<10:08,  2.80s/it]

wrote to /monoscene/Monoscene/outputs/NYU0212_0000.pkl


  1%|          | 2/218 [00:04<08:42,  2.42s/it]

wrote to /monoscene/Monoscene/outputs/NYU0302_0000.pkl


  1%|▏         | 3/218 [00:05<07:38,  2.13s/it]

...

wrote to /monoscene/Monoscene/outputs/NYU1163_0000.pkl


  7%|▋         | 15/218 [00:23<05:05,  1.50s/it]

wrote to /monoscene/Monoscene/outputs/NYU0207_0000.pkl


  7%|▋         | 16/218 [00:25<05:00,  1.49s/it]

wrote to /monoscene/Monoscene/outputs/NYU1399_0000.pkl
